In [9]:
import torch 
import torch.nn as nn

# input_size=256
# #midlayer_size=16
# encoding_dim=10
class Autoencoder(nn.Module):
    def __init__(self,input_size,encoding_dim):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size,16),
            nn.LeakyReLU(0.2,True),
            nn.Dropout(0.1),
            nn.Linear(16,14),
            nn.LeakyReLU(0.2,True),
            nn.Dropout(0.1),
            nn.Linear(14,12),
            nn.LeakyReLU(0.2,True),
            nn.Linear(12,encoding_dim),
            
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim,12),
            nn.LeakyReLU(0.2,True),
            nn.Dropout(0.1),
            nn.Linear(12,14),
            nn.LeakyReLU(0.2,True),
            nn.Dropout(0.1),
            nn.Linear(14,16),
            nn.LeakyReLU(0.2,True),
            nn.Linear(16,input_size),
            # nn.Tanh()
        )
    

    def forward(self,x):
        encode = self.encoder(x)
        
        regenerate = self.decoder(encode)
        
        return encode, regenerate
# autoencoder= Autoencoder(256,10)
# print(autoencoder)

In [2]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 842.6 kB/s eta 0:00:0000:0100:01


In [10]:
import pandas as pd
import seaborn as sns
df = pd.read_csv("intern_public_dataset_2022/df_train_20220802.csv", low_memory=False)
df_rt = pd.read_csv("intern_public_dataset_2022/df_rt_20220802.csv", low_memory=False)
df_test_unlabeled = pd.read_csv("intern_public_dataset_2022/df_test_unlabeled_20220802.csv", low_memory=False)
df["request_time"] = pd.to_datetime(df["request_time"])
df_test_unlabeled["request_time"] = pd.to_datetime(df_test_unlabeled["request_time"])
df_rt["request_time"] = pd.to_datetime(df_rt["request_time"])
df

,req_id,request_time,os_id,ssp_uid,slot_id,n_slot_position,sponsor_id,campaign_id,creative_id,region,...,prob_man,sponsor_fq,sponsor_inview_fq,sponsor_recency,sponsor_inview_recency,click_time,cv_time,click_flg,cv_flg,site_id
0,0,2022-06-14 15:00:07+00:00,Windows,0,0,3.0,0,0,0,aichi,...,0.985029,6,5,370,410,NaN,NaN,0,0,0
1,1,2022-06-14 15:00:07+00:00,iOS,1,1,NaN,1,1,1,osaka,...,NaN,0,0,-1,-1,NaN,NaN,0,0,1
2,2,2022-06-14 15:00:07+00:00,iOS,1,2,NaN,1,1,2,osaka,...,NaN,0,0,-1,-1,NaN,NaN,0,0,1
3,3,2022-06-14 15:00:08+00:00,Windows,2,3,NaN,2,2,3,tokyo,...,NaN,5,2,5,2,NaN,NaN,0,0,2
4,4,2022-06-14 15:00:09+00:00,Android,3,4,NaN,3,3,4,tokyo,...,NaN,0,0,-1,-1,NaN,NaN,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038562,5038562,2022-07-26 14:59:52+00:00,Android,158132,979,NaN,8,8,118,miyazaki,...,NaN,8,1,145,15356,NaN,NaN,0,0,11
5038563,5038563,2022-07-26 14:59:55+00:00,Android,164623,225,3.0,7,7,35,tokyo,...,NaN,2,0,37590,-1,NaN,NaN,0,0,31
5038564,5038564,2022-07-26 14:59:57+00:00,iOS,3054,2,NaN,2,2,708,kagoshima,...,NaN,3,1,219,308,NaN,NaN,0,0,1
5038565,5038565,2022-07-26 14:59:57+00:00,Android,53986,6,1.0,17,17,491,osaka,...,NaN,1,0,5,-1,NaN,NaN,0,0,5


In [4]:
dic_osid = []
dic_region= []
encode_cols = ['os_id','region']
for c in encode_cols:
    if c == 'os_id':
        env,obj = pd.factorize(df[c],sort=True)
        df[c] = env
        dic_osid = obj
    else:
        env,obj = pd.factorize(df[c],sort=True)
        df[c] = env
        dic_region = obj
print(dic_osid)
print(dic_region)
df

Index(['Android', 'Mac', 'Windows', 'iOS'], dtype='object')
Index(['aichi', 'akita', 'aomori', 'chiba', 'ehime', 'fukui', 'fukuoka',
       'fukushima', 'gifu', 'gumma', 'hiroshima', 'hokkaido', 'hyogo',
       'ibaraki', 'ishikawa', 'iwate', 'kagawa', 'kagoshima', 'kanagawa',
       'kochi', 'kumamoto', 'kyoto', 'mie', 'miyagi', 'miyazaki', 'nagano',
       'nagasaki', 'nara', 'niigata', 'oita', 'okayama', 'okinawa', 'osaka',
       'saga', 'saitama', 'shiga', 'shimane', 'shizuoka', 'tochigi',
       'tokushima', 'tokyo', 'tottori', 'toyama', 'wakayama', 'yamagata',
       'yamaguchi', 'yamanashi'],
      dtype='object')


,req_id,request_time,os_id,ssp_uid,slot_id,n_slot_position,sponsor_id,campaign_id,creative_id,region,...,prob_man,sponsor_fq,sponsor_inview_fq,sponsor_recency,sponsor_inview_recency,click_time,cv_time,click_flg,cv_flg,site_id
0,0,2022-06-14 15:00:07+00:00,2,0,0,3.0,0,0,0,0,...,0.985029,6,5,370,410,NaN,NaN,0,0,0
1,1,2022-06-14 15:00:07+00:00,3,1,1,NaN,1,1,1,32,...,NaN,0,0,-1,-1,NaN,NaN,0,0,1
2,2,2022-06-14 15:00:07+00:00,3,1,2,NaN,1,1,2,32,...,NaN,0,0,-1,-1,NaN,NaN,0,0,1
3,3,2022-06-14 15:00:08+00:00,2,2,3,NaN,2,2,3,40,...,NaN,5,2,5,2,NaN,NaN,0,0,2
4,4,2022-06-14 15:00:09+00:00,0,3,4,NaN,3,3,4,40,...,NaN,0,0,-1,-1,NaN,NaN,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038562,5038562,2022-07-26 14:59:52+00:00,0,158132,979,NaN,8,8,118,24,...,NaN,8,1,145,15356,NaN,NaN,0,0,11
5038563,5038563,2022-07-26 14:59:55+00:00,0,164623,225,3.0,7,7,35,40,...,NaN,2,0,37590,-1,NaN,NaN,0,0,31
5038564,5038564,2022-07-26 14:59:57+00:00,3,3054,2,NaN,2,2,708,17,...,NaN,3,1,219,308,NaN,NaN,0,0,1
5038565,5038565,2022-07-26 14:59:57+00:00,0,53986,6,1.0,17,17,491,32,...,NaN,1,0,5,-1,NaN,NaN,0,0,5


In [5]:
df = df.fillna(df.mean())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [6]:
df['request_time']=pd.to_datetime(df['request_time'])
new = pd.DataFrame((df.groupby([(df['request_time'].dt.month),(df['request_time'].dt.day)])["click_flg"].sum())/(df.groupby([(df['request_time'].dt.month),(df['request_time'].dt.day)])["click_flg"].size()))


In [7]:
df.dropna(axis=1,how='any',inplace=True)
df

,req_id,request_time,os_id,ssp_uid,slot_id,n_slot_position,sponsor_id,campaign_id,creative_id,region,ref,prob_man,sponsor_fq,sponsor_inview_fq,sponsor_recency,sponsor_inview_recency,click_flg,cv_flg,site_id
0,0,2022-06-14 15:00:07+00:00,2,0,0,3.000000,0,0,0,0,0.000000,0.985029,6,5,370,410,0,0,0
1,1,2022-06-14 15:00:07+00:00,3,1,1,2.264248,1,1,1,32,124154.795406,0.525834,0,0,-1,-1,0,0,1
2,2,2022-06-14 15:00:07+00:00,3,1,2,2.264248,1,1,2,32,124154.795406,0.525834,0,0,-1,-1,0,0,1
3,3,2022-06-14 15:00:08+00:00,2,2,3,2.264248,2,2,3,40,2.000000,0.525834,5,2,5,2,0,0,2
4,4,2022-06-14 15:00:09+00:00,0,3,4,2.264248,3,3,4,40,3.000000,0.525834,0,0,-1,-1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038562,5038562,2022-07-26 14:59:52+00:00,0,158132,979,2.264248,8,8,118,24,124154.795406,0.525834,8,1,145,15356,0,0,11
5038563,5038563,2022-07-26 14:59:55+00:00,0,164623,225,3.000000,7,7,35,40,124154.795406,0.525834,2,0,37590,-1,0,0,31
5038564,5038564,2022-07-26 14:59:57+00:00,3,3054,2,2.264248,2,2,708,17,1834.000000,0.525834,3,1,219,308,0,0,1
5038565,5038565,2022-07-26 14:59:57+00:00,0,53986,6,1.000000,17,17,491,32,5.000000,0.525834,1,0,5,-1,0,0,5


In [8]:
X = df.drop(['click_flg','request_time','cv_flg'], axis=1)
Y = df['click_flg']
X.to_csv('X_nonan.csv',index=False)
Y.to_csv('Y_nonan.csv',index=False)


In [12]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=0, stratify=Y)

In [23]:
len(X_train.values[0])

16

In [19]:
df_dataset = [X_train, X_test, Y_train, Y_test]
dataset = []
for i in range(len(df_dataset)):
    tmp = torch.Tensor(df_dataset[i].values)
    dataset.append(tmp)
dataset

[tensor([[ 7.4531e+05,  3.0000e+00,  1.4602e+04,  ...,  1.6400e+02,
           2.8360e+03,  7.4000e+01],
         [ 2.1903e+06,  0.0000e+00,  1.5258e+05,  ...,  9.7000e+01,
           9.4000e+01,  2.7000e+01],
         [ 3.5654e+06,  2.0000e+00,  1.9060e+03,  ...,  3.0930e+03,
           1.8865e+04,  1.0000e+01],
         ...,
         [ 9.3283e+04,  3.0000e+00,  2.6729e+04,  ...,  1.4000e+01,
          -1.0000e+00,  1.0000e+00],
         [ 2.6328e+06,  3.0000e+00,  1.1396e+05,  ...,  3.6000e+01,
           3.3000e+01,  5.2000e+01],
         [ 4.3728e+06,  3.0000e+00,  2.7851e+04,  ...,  5.0000e+00,
          -1.0000e+00,  1.0000e+00]]),
 tensor([[ 4.5655e+06,  0.0000e+00,  2.0897e+05,  ..., -1.0000e+00,
          -1.0000e+00,  2.3000e+01],
         [ 2.7720e+06,  0.0000e+00,  5.5524e+04,  ...,  5.7300e+02,
          -1.0000e+00,  5.0000e+00],
         [ 4.6009e+06,  0.0000e+00,  1.8214e+05,  ...,  1.4310e+04,
           1.4490e+04,  2.1000e+01],
         ...,
         [ 4.4862e+06,  3

In [25]:

  
import random
import torch
import torch.nn as nn


# sigmoid=nn.Sigmoid()

# random.seed(0)
# d = []
# e = []
# for i in range(10000):
#     r =  [float(random.randint(-10,10)) for i in range(20)]
#     d.append(r)
#     e.append(r)

# def generatex(num):
#     return 2*num+num*num

# def formular(x):
#     return 2*x*x-x*x*x*x+np.log(1+2*x*x)+np.exp(1-x)


# for j in range(1,10000):
#     tmp = []
#     for i in range(1,21):
#         t = generatex(i/j)
#         tmp.append(t)

#     d.append(tmp)
#     e.append(tmp)





device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# device = "cpu"

autoencoder = Autoencoder(16,10).to(device)

x = dataset[0].to(device)
y = dataset[0].to(device)
x.requires_grad = True
y.requires_grad = True



#print(d)

#print(type(d))

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

# x = torch.unsqueeze(torch.linspace(-1,1,100),dim=)
# y = x.pow(3)+0.1*torch.randn(x.size())
# print(x)


optimizer = torch.optim.SGD(autoencoder.parameters(),lr = 0.5)
loss_func = torch.nn.MSELoss()


for t in range(5000):
    prediction = autoencoder(x)
    loss = loss_func(prediction[1],y)
    if t%1000==0:
        # print(prediction[1])
        # print(y)

        print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# print(x.device,y.device)





# testx=[]
# testy=[]

# c = []
# for i in range(1,21):
#     t = generatex(i/10002)
#     c.append(t)

# testx.append(c)
# testy.append(c)
# testx = torch.Tensor(testx).to(device)
# testy = torch.Tensor(testy).to(device)
# with torch.no_grad():
#     prediction = autoencoder(testx)

# print(testx)
# print(prediction[0])
# print(prediction[1])

tensor(5.3149e+11, grad_fn=<MseLossBackward0>)
tensor(5.3149e+11, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 